Goal: Create agent that can help to write high quality blogpost and having agent to review the output 

# Setup

In [1]:
# define llm 
llm_config = {"model": "gpt-3.5-turbo"}

# The task

In [2]:
task = '''
        Write a concise but engaging blogspot about
        Deeplearning.AI. Make sure the blogspot is within 100 words 
        '''

# Create a writer agent 

In [3]:
import autogen 

# create writer agent 
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise"
    "blogspot (with title) on given topics. You must polish your."
    "Writing based on the feedback you receive and give a refined version"
    "Only return your fina work without addition comments.",
    llm_config=llm_config
)

In [4]:
# reply function, and input the task 
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
# print result 
print(reply)

Title: "Unveiling the Power of Deeplearning.AI"

Unleash the transformative potential of AI with Deeplearning.AI! Offering cutting-edge courses in deep learning, Andrew Ng's platform equips learners with the skills to thrive in the AI era. From neural networks to machine learning algorithms, Deeplearning.AI demystifies complex concepts through engaging content and hands-on projects. Whether you're a novice or seasoned professional, this platform caters to all skill levels. Join a vibrant community of learners worldwide and embark on a journey of innovation and discovery. Elevate your career and unleash the power of AI with Deeplearning.AI today!


# Adding Reflection
- Create a critic agent to reflect on the work of the writer agent 

In [6]:
# agent to give critic for the writer output 
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config = llm_config,
    system_message="You are a critic. You review the work of "
    "The writer and provide constructive"
    "feedback to help improve the quality of the content."
)

In [7]:
# initiates interaction between agents 
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogspot about
        Deeplearning.AI. Make sure the blogspot is within 100 words 
        

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of Deep Learning with Deeplearning.AI

Dive into the world of AI with Deeplearning.AI, a leading platform revolutionizing the way we understand technology. Offering cutting-edge courses curated by experts like Andrew Ng, Deeplearning.AI equips learners with the skills to build AI systems that can interpret data, recognize patterns, and make informed decisions. Whether you're a beginner or an experienced professional, Deeplearning.AI offers a comprehensive learning experience that's both immersive and rewarding. Join today, and unlock your potential in the dynamic field of deep learning. Harness the power of AI with Deeplearning.AI and shape the future with innovation.

----------------------------------------

# Nested Chat 
- Having 4 agent to review the blog post and each agent having specialized task 

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer known for your ability to optimize content for search angines"
    "Ensuring that it ranks well and attartcs organic traffic."
    "Make sure your suggestion is consie (within 3 bullet points) concrete and to the point."
    "Begin the review by stating your role."
)

In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer known for your ability to ensure that content is legally compliant"
    "and free from any potential legal issues."
    "Make sure your suggestion is concise (within 3 bullet point) concrete and to the point"
    "Begin the review by stating your role"
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for your ability to ensure that content is ethically sound"
    "and free from any potential ethical issues"
    "Make sure your suggestion is concise (within 3 bullet point) concrete and to the point"
    "Begin the review by stating your role"
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review the work of other reviewers and give a final suggestion on the content."
)

# Orchestrate the nested chats to solve the task 

In [16]:
def reflection_message(recipient, messages, sender, config):
    """
    Generates a reflection message for the recipient based on the last chat message.
    
    Returns:
    - A string containing a message that asks the recipient to review the content 
      of the last message sent by the sender.
    """
    
    # Retrieve the last message content from the recipient's chat history
    last_message_content = recipient.chat_messages_for_summary(sender)[-1]['content']
    
    # Format and return the review request message, including the last message content 
    return f''' Review the following content.
            \n\n {last_message_content}'''
            
            
# Define a list of review chat tasks to be executed -> used as critic 
review_chats = [
   {
     # The SEO Review agent responsible for the review 
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {
        "summary_prompt" : 
           "Return review into as JSON object only: {'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
      },
     "max_turns": 1
   },
    
   {
      # The legal review agent responsible for the review 
      "recipient": legal_reviewer, 
      "message": reflection_message, 
      "summary_method": "reflection_with_llm",
      "summary_args": {
         "summary_prompt" : "Return review into as JSON object only:{'Reviewer': '', 'Review': ''}.",
      },
      "max_turns": 1
    },
   
   {
      # The ethics review agent responsible for the review 
      "recipient": ethics_reviewer, 
      "message": reflection_message, 
      "summary_method": "reflection_with_llm",
      "summary_args": {
         "summary_prompt" : "Return review into as JSON object only: {'reviewer': '', 'review': ''}",
      },
      "max_turns": 1
   },
   
   {
      # The meta review agent responsible for the aggregating feedback 
      "recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
      "max_turns": 1},
]


In [13]:
# registers a set of nested chat "review_chats" with critic agent 
critic.register_nested_chats(
    review_chats,
    trigger=writer, # The trigger for the nested chats
)

In [14]:
# initiate a chat between agent (critic and writer)
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogspot about
        Deeplearning.AI. Make sure the blogspot is within 100 words 
        

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of Deep Learning with Deeplearning.AI

Dive into the world of AI with Deeplearning.AI, a leading platform revolutionizing the way we understand technology. Offering cutting-edge courses curated by experts like Andrew Ng, Deeplearning.AI equips learners with the skills to build AI systems that can interpret data, recognize patterns, and make informed decisions. Whether you're a beginner or an experienced professional, Deeplearning.AI offers a comprehensive learning experience that's both immersive and rewarding. Join today, and unlock your potential in the dynamic field of deep learning. Harness the power of AI with Deeplearning.AI and shape the future with innovation.

----------------------------------------

d:\MINICONDA\envs\autogen_env\Lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

As a legal reviewer, my suggestions for the provided content are:

- Ensure compliance with intellectual property laws by verifying if any copyrighted material is being used without permission.
- Check for proper use of trademarked terms such as "Deeplearning.AI" to avoid potential infringement issues.
- Confirm that any claims made about the platform's offerings are accurate and supported by evidence to prevent false advertising accusations.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

 Review the following content.
            

 Title: Unveiling the Power of Deep Learning with Deeplearning.AI

Dive into the world of AI with Deeplearning.AI, a leading platform revolutionizing the way we understand technol

In [15]:
# Get the summary 
print(res.summary)

Title: Unleashing the Power of Deeplearning.AI's Educational Platform

Embark on an exhilarating journey into the realm of AI with Deeplearning.AI. Lead by industry expert Andrew Ng, this educational platform offers a gateway to unlocking the potential of deep learning. From understanding complex algorithms to building intelligent systems, Deeplearning.AI provides a comprehensive learning experience suitable for all skill levels. Whether you are a novice enthusiast or a seasoned professional, the curated courses will equip you with the skills needed to thrive in the ever-evolving field of artificial intelligence. Join the AI revolution with Deeplearning.AI and shape a future driven by innovation and knowledge.
